## Imports

In [2]:
from booster_robotics_sdk_python import B1LocoClient, ChannelFactory, RobotMode, B1HandIndex, GripperControlMode, Position, Orientation, Posture, GripperMotionParameter, GetModeResponse, Quaternion, Frame, Transform, DexterousFingerParameter
import time

## Control

In [13]:
## Establish Connection 
network_interface = "192.168.1.53"
ChannelFactory.Instance().Init(0, network_interface)

client = B1LocoClient()
client.Init()

In [ ]:
yaw, pitch = 0.0, 1.0
need_print = True
res = client.RotateHead(pitch, yaw)
print(f"Rotate head result: {res}")

Change to Prepare mode: 100


# Actions

In [ ]:
SwitchHandEndEffectorControlMode(False)

In [ ]:
# Modes
res = client.ChangeMode(RobotMode.kPrepare)
res = client.ChangeMode(RobotMode.kDamping)
res = client.ChangeMode(RobotMode.kWalking)
res = client.ChangeMode(RobotMode.kCustom)

In [ ]:
# Head
res = client.RotateHead(pitch, yaw)

In [ ]:
# Arm
tar_posture = Posture()\
tar_posture.position = Position(0.35, 0.25, 0.1)
tar_posture.orientation = Orientation(-1.57, -1.57, 0.0)
res = client.MoveHandEndEffectorV2(
    tar_posture, 2000, B1HandIndex.kLeftHand)


In [ ]:
# Movement
x, y, z = 0.0, -0.2, 0.0
res = client.Move(x, y, z)